# NSA

This notebook is based on the brilliant book by Andreas Eschbach: ["NSA: Nationales Sicherheitsamt"](https://www.amazon.de/dp/B07D18P88V/). 


The core idea of that book: imagine computers, the internet, and mobile phones developed roughly 70 years earlier, namely in the beginning of the 20th century. A special office called "Nationales Sicherheitsamt (NSA)" collects and analyzes data to identify potential "risks" for the nation. In the 1930s, the nazi regime takes control of the government and the NSA. Hence they get access to all that data and look for ways to abuse that data for their goals, especially the tracing of hidden jews and political opponents.

In this notebook, we demonstrates the scenario described on the first 42 pages.

**Notice**: This notebook based on Eschbachs fantastic book is a **warning**: seemingly useless/junk data when combined in the right way, can yield shocking insights. 

This scenario is **yet another example of the highly unpredictable "big data arithmetic": Joining apples with oranges may yield anything.**

And you do not need any machine learning or artificial intelligence for this. SQL is enough, and even a relatively restricted subset of SQL (SQL 92), as supported by sqlite3 and used in this notebook, is enough in this case.

Copyright Jens Dittrich & Christian Schön, [Big Data Analytics Group](https://bigdata.uni-saarland.de/), [CC-BY-SA](https://creativecommons.org/licenses/by-sa/4.0/legalcode)

## Load Data

Before we can start analyzing the data, we first have to load data, which I already imported from the corresponding csv files into an appropriate database schema. This is fake data.

In [ ]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

!wget -qOnsa.db 'https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=nsa.db'

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%load_ext sql

# Verbinde Dich zu einer in - Memory Datenbank
%sql sqlite:///nsa.db
%sql PRAGMA foreign_keys = ON

In [ ]:
%sql SELECT * from purchases limit 10

The fake data consists of 15 households with 40 official inhabitants. For the purchases, each (adult) inhabitant can choose out of 33 different articles of food. The corresponding table covers data from 183 days (1942-04-01 to 1942-09-30). The example assumes that all analysis steps are done on the 5th of october 1942, as in the book by Andreas Eschbach.

## Show Schema of Table Citizens

The command ".schema table_name" outputs the SQL command used to generate this table. You can use it to determine the names of the attributes of a table and their datatypes. 

In [ ]:
%%bash
echo .schema households | sqlite3 nsa.db

In [ ]:
%%bash
echo .schema livingIn | sqlite3 nsa.db

In [ ]:
%sql SELECT * FROM livingin LIMIT 10;

In [ ]:
%sql SELECT * FROM citizens LIMIT 10;

In [ ]:
%sql SELECT * FROM households LIMIT 10;

## Citizens & Households

In a first step, we want to show the citizens and the households they currently live in. The current household can be determined by looking at the "until" attribute. If it is the empty string (remember: the fields are generated with datatype "Text"), the citizen currently lives in this household. If it is non-empty, the citizen lived here in the past.

In [ ]:
%%sql
--- join three tables citizens, livingIn, and households:
SELECT citizens.firstname, citizens.lastname, households.street, households.postcode, households.city
FROM citizens
    JOIN livingIn ON citizens.id = livingIn.citizen_id
    JOIN households ON households.id = livingIn.household_id
WHERE livingIn.until IS ""
LIMIT 10;

## Count Inhabitants

To search for hidden persons, we first need to know the number of (official) inhabitants of each household. This can be achieved by grouping over the id of the household and counting the number of citizen_id within each group.

## Task 1.1
Create a view `inhabitantsPerHousehold` with attributes `household_id` and `numInhabitants`. 

In [ ]:
%%sql
--- display the result:
SELECT *
FROM inhabitantsPerHousehold

## Task 1.2

For each household also show the corresponding address by joining the `household` table.

## Count Calories Per Household

For simplicity, all food articles are measured in kilogram or litres. For simplicity we assume that one litre is equal to one kilogram. The nutritional values are given in kcal per 100g. We further assume that all food purchases are consumed within the household of the citizen who bought it, i.e. there is no food sharing between households or similar things.

## Task 2.1

Define a view `caloriesPerHouseholdAndMonth` computing the number of calories purchased for each houshold and month. Notice that we have to extract the month from each date using the function `strftime('%m', purchases.date)`. Hence, create a grouping based on household and month, and use `SUM` to get the total calories of that group. 

In [ ]:
%%sql 
--- display the result:
SELECT *
FROM caloriesPerHouseholdAndMonth LIMIT 10;

We multiply by 10 as the nutritional values are stored per 100g, however the items purchased are stored by 1000g.

## Task 2.2

Define a view `caloriesPerHousehold` ignoring the individual months and computing aggregates for the entire time period:

In [ ]:
%sql SELECT * FROM caloriesPerHousehold

Show total rounded calories per household for all available data in descending order:

In [ ]:
%%sql
SELECT caloriesPerHousehold.household_id, ROUND(caloriesPerHousehold.calories, 5) AS totalCalories
FROM caloriesPerHousehold
ORDER BY caloriesPerHousehold.calories DESC;

The pure amount of calories per household is however not meaningful for the task specified above. A large household with 5 or 6 inhabitants will by nature have a much larger consumption as a small household with only 1 or 2 inhabitants. The next step is therefore to compute the average amount of calories that each houshold consumes per day and inhabitant. 

## Daily Calories Per Inhabitant & Household

We reuse the views defined before. The daily calories per household and inhabitant are computed by dividing the total calories of the household by the number of inhabitants and the number of days (183). As we are interested in households consuming on average much more than expected, we are ordering the results in decreasing order based on the average amount of calories.

## Task 3.1 

Create a Statement, shows that data, hence select 
- household_id
- numInhabitants
- daily calories (total calories divided by 183)
- daily calories per inhabitant

and order as specified above.

## Daily Calories Per Inhabitant

To detect outliers, we need to know the average daily amount of calories consumed by each inhabitant of our city.

In [ ]:
%%sql
DROP VIEW IF EXISTS totalCalories;

CREATE VIEW totalCalories AS
    SELECT SUM(caloriesPerHousehold.calories) AS calories
    FROM caloriesPerHousehold;
    
SELECT * FROM totalCalories;

In [ ]:
%%sql
DROP VIEW IF EXISTS totalInhabitants;

CREATE VIEW totalInhabitants AS
    SELECT SUM(inhabitantsPerHousehold.numInhabitants) AS numInhabitants
    FROM inhabitantsPerHousehold;

SELECT * FROM totalInhabitants;

## Task 3.2 

Define a view `averageCaloriesPerInhibitant` that produces the average Number of calories `dailyCaloriesPerInhabitant` as single attribute.

In [ ]:
%%sql
SELECT *
FROM averageCaloriesPerInhibitant;

Notice that this average is counted assuming 40 inhabitants where in fact there are more (hidden inhabitants). Hence this average is very likely too high.

## Detecting Outliers

The number of calories each person consumes within each day is not strictly constant, but depends on age, gender and other factors such as physical work. For simplicity, we will assume for this exercise that each person consumes roughly 2500 kcal per day.

Analyzing the results of the previous queries, we clearly see that the household with id 7 consumes 7125 kcal per day and inhabitant, nearly three times the expected value. Considering the fact that this household has only one official inhabitant, we can expect two hidden persons in this household.

For the household with id 9, we also see an increased consumption with roughly 3150 kcal per day and inhabitant. This is clearly more than expected. Considering the fact that this household has 3 official inhabitants, we would expect a consumption of roughly 7500 kcal per day for the complete household. However, we measured a consumption of 9451 kcal which is nearly the expected consumption of 4 persons. Based on the data, there is a high chance that one person is hidden in this household.

For the remaining households, we only find slight deviations from the expected 2500 kcal per day and inhabitant. Values of less than 2700 kcal per day and inhabitant do not necessarily indicate the presence of hidden persons, but could also result from people having a hard physical work and therefore eating more than the average person.

## Beyond the Book: More Advanced Analysis Techniques

Consider the following scenario: Based on lists of all inhabitants of Saarbrücken and their movements (some might have emigrated), you can identify persons which are hiding somewhere in the city. For the sake of simplicity, we assume that you know that (at the moment of this analysis) 6 persons are still missing and must live in one of the 15 households contained in our database.

Based on the previous analysis steps, you already identified three of them. However, there are still three persons missing and you know for sure they did not leave the city and they do not receive food from somewhere outside these 15 households. The previous analysis steps clearly show that each of the other households more or less exactly consumes the expected amount of calories. How can these persons still hide in the city, apparently without eating?

### Considering Age

So far, we treated all persons equal, no matter which age or gender they have. However, this is not true in reality. A baby of less than two years will not consume a measurable amount of solid food bought by its parents. We should therefore exclude small babies when counting the inhabitants of each household. Remember: As we are following the example in the book, our analysis takes place on the 5th of october 1942. We are therefore looking for inhabitants born before the 5th of october 1940.

## Task 4.1

Define a view `inhabitantsPerHouseholdExBabies` where we exclude inhibitants younger than two years. We assume that this analysis is done on Oct 5th, 1942 (db format `"1942-10-05 00:00:00"`). Birthday information is in the `citizens` table.

In [ ]:
%sql SELECT sum(numInhabitants) FROM inhabitantsPerHouseholdExBabies

## Task 4.2

Run the same calories analysis as above, however, this time excluding inhabitants younger than two:

For the household with id 11, we see a dramatic change in average calories consumed. In the previous analysis step, we observed a value of 2508 kcal per inhabitant and day which we considered normal. Now we see a dramatic rise up to 3762 kcal per inhabitant and day. We would expect a consumption of roughly 5000 kcal per day for the complete household, maybe adding a few hundred kcal for the baby. However, we now measure roughly 7524 kcal for the complete household which can only be explained by a hidden person living there.